<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
# Replace the entire previous cell with this one to force CPU mode and retry downloads safely.

import os
import shutil
import subprocess
from pathlib import Path

# clear problematic envs
os.environ.pop('PYTHONPATH', None)
os.environ.pop('MPLBACKEND', None)

# Force processes to see no GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Clone / change dir if needed (keeps original behaviour)
if Path.cwd().name != 'text-generation-webui':
    print("\n --> Installing the web UI (CPU mode). This will take a while.\n")
    if not Path("text-generation-webui").exists():
        subprocess.run("git clone https://github.com/gitleon8301/MY-AI-Gizmo-working", shell=True, check=True)
    os.chdir("text-generation-webui")
    # Install the project in an isolated environment (do not auto-launch)
    # Keep LAUNCH_AFTER_INSTALL=FALSE so installation finishes without starting GPU-specific runtime
    subprocess.run('GPU_CHOICE=A LAUNCH_AFTER_INSTALL=FALSE INSTALL_EXTENSIONS=FALSE ./start_linux.sh', shell=True, check=True)

# ---------- User-editable parameters ----------
model_url = "https://huggingface.co/google/gemma-2-9b-it-GGUF"
branch = ""   # if unsure, set to "" or "main"
# CPU-only flags (remove all GPU-only/quant flags)
command_line_flags = "--cpu"
api = False
# ----------------------------------------------

if api:
    for param in ['--api', '--public-api']:
        if param not in command_line_flags:
            command_line_flags += f" {param}"

model_url = model_url.strip()
output_folder = ""
if model_url != "":
    if not model_url.startswith('http'):
        model_url = 'https://huggingface.co/' + model_url

    url_parts = model_url.strip('/').split('/')
    output_folder = f"{url_parts[-2]}_{url_parts[-1]}"
    branch = (branch or "").strip().strip('"\'')

    # Helper to run the download command and return CompletedProcess
    def run_download(cmd):
        print(f"\nRunning: {cmd}\n")
        return subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Attempt download with provided branch (if not blank/main), otherwise without branch
    downloaded = False
    tried_cmds = []
    if branch and branch not in ['', 'main']:
        tried_cmds.append(f"python download-model.py {model_url} --branch {branch}")
        proc = run_download(tried_cmds[-1])
        if proc.returncode == 0:
            downloaded = True
        else:
            # If error looks like a 416 or partial-file issue, clear HF cache and retry fallback
            stderr = proc.stderr.lower()
            if "416" in stderr or "range not satisfiable" in stderr or proc.returncode != 0:
                hf_cache = Path.home() / ".cache" / "huggingface" / "hub"
                if hf_cache.exists():
                    print("Detected download error. Clearing Hugging Face cache and retrying with 'main' / without --branch.")
                    shutil.rmtree(hf_cache)
                # try with main first
                tried_cmds.append(f"python download-model.py {model_url} --branch main")
                proc2 = run_download(tried_cmds[-1])
                if proc2.returncode == 0:
                    downloaded = True
                    branch = "main"
                else:
                    # final fallback: try without --branch
                    tried_cmds.append(f"python download-model.py {model_url}")
                    proc3 = run_download(tried_cmds[-1])
                    if proc3.returncode == 0:
                        downloaded = True
                        branch = ""
    else:
        tried_cmds.append(f"python download-model.py {model_url}")
        proc = run_download(tried_cmds[-1])
        if proc.returncode == 0:
            downloaded = True
        else:
            # try with main branch explicitly
            tried_cmds.append(f"python download-model.py {model_url} --branch main")
            proc2 = run_download(tried_cmds[-1])
            if proc2.returncode == 0:
                downloaded = True
                branch = "main"

    if not downloaded:
        print("\nAll automatic download attempts failed. Tried commands:")
        for t in tried_cmds:
            print("  ", t)
        print("\nYou can: (1) check that the branch exists on Hugging Face, (2) download the model manually from the repo page and place it in the models/ directory, or (3) try a different branch/name.")
    else:
        # adapt output_folder name to include branch if non-empty and not main
        if branch and branch not in ['', 'main']:
            output_folder += f"_{branch}"
        print(f"\nModel download succeeded (or model found). Using folder: {output_folder}")

# ---------- Start the web UI in CPU mode ----------
cmd = f"./start_linux.sh {command_line_flags} --share"
if output_folder:
    cmd += f" --model {output_folder}"

print("\nStarting the web UI (CPU mode). Command:")
print(cmd)
# Use exec so the process takes over the cell output (similar to original behaviour)
os.execvp("./start_linux.sh", ["./start_linux.sh"] + cmd.split()[1:])
